In [1]:
## Importing required Libraries 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder , OneHotEncoder
import pickle 

In [2]:
## Loading the Dataset 
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Data Preprocessing 

In [3]:
## Dropping unnecessary/irrelevant columns 
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)
## axis = 1 (column-wise)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Encoding : Converting categorical variables into numerical variables 
## "Geography" and "Gender" are categorical columns so converting them into numerical columns 

## In "Gender" Column we have only two categories Male/Female so we are applying Label Encoding 
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])

In [5]:
## In "Geography" Column we have more than two categories so we are applying One Hot Encoding 
ohe_encoder_geography = OneHotEncoder()
geo_encoder = ohe_encoder_geography.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
## getting features name from ohe_encoder_geography
feature_names = ohe_encoder_geography.get_feature_names_out()
feature_names

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
## converting to DataFrame
geography_df = pd.DataFrame(geo_encoder.toarray(),columns=feature_names)
geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
## Combining/Updating these columns with original data 
data = pd.concat([data,geography_df],axis=1)

## Dropping Gender Column 
data = data.drop(["Geography"],axis=1)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
## Saving the encoders as pickle file (for future use) 

with open("label_encoder_gender.pkl","wb") as file :
    pickle.dump(label_encoder_gender,file)

with open("ohe_encoder_geography.pkl","wb") as file :
    pickle.dump(ohe_encoder_geography,file)

In [10]:
## Dividing the dataset into Independent and Dependent features 
X = data.drop("Exited",axis=1) ## Independent Features
y = data["Exited"] ## Dependent Feature 

## Applying train_test_split 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Applying Standardization (on input features)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
## Saving the scaler file
with open("scaler.pkl","wb") as file :
    pickle.dump(scaler,file)

### Model Building and Training 

##### Training the Artificial Neural Network 

In [12]:
## Importing Tensorflow and Keras
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime 

In [13]:
print(X_train.shape) ## ( 8000 rows , 12 columns )
print(X_train.shape[1]) ## 12 (number of inputs)
print((X_train.shape[1],1)) ## input shape

(8000, 12)
12
(12, 1)


In [14]:
## Building ANN (Sequential Model) from scratch 

model = Sequential(
    [
        Dense(64,activation="relu",input_shape=(X_train.shape[1],)), ## Hidden Layer 1 connected with input layer
        Dense(32,activation="relu"), ## Hidden Layer 2 connected to HL1 (no need to given input in this case as it is sequential/connected)
        Dense(1,activation="sigmoid") ## Output Layer
    ]
)

In [15]:
## Summary of our Model 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
## Defining Optimizers 
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

## Defining Loss 
loss = tf.keras.losses.BinaryCrossentropy()

In [17]:
## Simple way of compiling the model 
## Compilation is done to apply Forward and Backward propagation
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])

In [18]:
## Setting-up TenserBoard and log directory 
## TensorBoard - Visualization 
## log directory - to store training information
log_directory = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_directory,histogram_freq=1)

In [19]:
## Setting-up Early Stopping
## Early Stopping is done , because our model is sometimes trained in few no. of repetitions(epochs) so we could stop it
## main aim to prevent overfitting and conserve computational resources during model training 
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)

In [20]:
## Training the Model 
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 7ms/step - loss: 0.4511 - accuracy: 0.8061 - val_loss: 0.3935 - val_accuracy: 0.8315
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3827 - accuracy: 0.8419 - val_loss: 0.3552 - val_accuracy: 0.8570
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3535 - accuracy: 0.8546 - val_loss: 0.3463 - val_accuracy: 0.8590
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3431 - accuracy: 0.8605 - val_loss: 0.3435 - val_accuracy: 0.8625
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3381 - accuracy: 0.8575 - val_loss: 0.3450 - val_accuracy: 0.8640
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3350 - accuracy: 0.8610 - val_loss: 0.3392 - val_accuracy: 0.8645
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3319 - accuracy: 0.8626 - val_loss: 0.3415 - val_accuracy: 0.86

###### We have set epochs = 100 but it has stopped at epochs = 19 , because our model is well-trained at 19 epochs only  

In [21]:
## Saving our model 
model.save("model.h5")

f:\Deep Learning Projects\Churn Modelling using ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
## Load TensorBoard Extension using magic functions
%load_ext tensorboard

In [ ]:
%tensorflow --logdir log/fit

In [29]:
## Loading the saved files 

## Load the ANN trained model
from tensorflow.keras.models import load_model
model = load_model("model.h5")

## Load the encoder and scaler file

with open("label_encoder_gender.pkl","rb") as file :
    label_encoder_gender = pickle.load(file)

with open("ohe_encoder_geography.pkl","rb") as file :
    ohe_encoder_geography = pickle.load(file)

with open("scaler.pkl","rb") as file :
    scaler = pickle.load(file)

#### Prediction using the trained ANN Model